In [1]:
# train.py
from copy import deepcopy
import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions import inference, fitness_fn, generation, generation_valid, generation_test

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection
from Evolution import Evolution

In [2]:
# Load Data
data_1m = load_data_1m('/root/daily/bit/data/1min_bitusdt.pkl')
data_1m = data_1m.iloc[:100000]

# Resample data to 1D
data_1d = resample_data(data_1m, '1D')
data_1d['Close time'] = data_1d.index
data_1d = data_1d.reset_index(drop=True)

# Apply Feature Calculations
# For 1D Data
data_1d, ma_cols_1d, ma_cols_rel_1d = calculate_MA_data(data_1d, 60, 'EMA', '_1d')
data_1d, bb_cols_1d, bb_cols_rel_1d = calculate_ema_bollinger_bands(data_1d, 60, extra_str='_1d')
data_1d, rsi_cols_1d = calculate_rsi(data_1d, window=20, extra_str='_1d')
data_1d, macd_cols_1d = calculate_macd(data_1d, 20, 120, 60, extra_str='_1d')
data_1d, stoch_cols_1d = calculate_stochastic_oscillator(data_1d, 60, 20, extra_str='_1d')
data_1d, adx_cols_1d = calculate_adx(data_1d, 60, extra_str='_1d')
data_1d, atr_cols_1d = calculate_atr(data_1d, 60, extra_str='_1d')
data_1d, obv_cols_1d = calculate_obv(data_1d, extra_str='_1d')
data_1d, will_cols_1d = calculate_williams_r(data_1d, 60, extra_str='_1d')
data_1d, base_feature_1d = base_feature_fn(data_1d, extra_str='_1d')
data_1d, cyclice_encoding_1d = cyclic_encode_fn(data_1d, 'Close time', 'day_of_year')

# For 1M Data
data_1m, ma_cols, ma_cols_rel = calculate_MA_data(data_1m, 240, 'EMA')
data_1m, bb_cols, bb_cols_rel = calculate_ema_bollinger_bands(data_1m, 240)
data_1m, rsi_cols = calculate_rsi(data_1m, window=60)
data_1m, macd_cols = calculate_macd(data_1m, 60, 600, 240)
data_1m, stoch_cols = calculate_stochastic_oscillator(data_1m, 240, 60)
data_1m, adx_cols = calculate_adx(data_1m, 240)
data_1m, atr_cols = calculate_atr(data_1m, 240)
data_1m, obv_cols = calculate_obv(data_1m)
data_1m, will_cols = calculate_williams_r(data_1m, 240)
data_1m, base_feature = base_feature_fn(data_1m)
data_1m, cyclice_encoding = cyclic_encode_fn(data_1m, 'Open time')

# Prepare Feature Columns
drop_column = [
    'Open time', 'Close time', 'Quote asset volume', 'Ignore',
    'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume'
]
feature_column = (
    ma_cols_rel + bb_cols_rel + rsi_cols + macd_cols + stoch_cols +
    adx_cols + will_cols + base_feature + cyclice_encoding  # Excluding obv and atr
)
feature_column_1d = (
    ma_cols_rel_1d + bb_cols_rel_1d + rsi_cols_1d + macd_cols_1d + stoch_cols_1d +
    adx_cols_1d + will_cols_1d + base_feature_1d + cyclice_encoding_1d
)

# Apply Log Transform
for feature in feature_column:
    data_1m[feature] = log_transform(data_1m[feature])

for feature in feature_column_1d:
    data_1d[feature] = log_transform(data_1d[feature])

# Generate Entry Positions
entry_pos_list, patience_list = BB_fitness_fn(data_1m)

# Prepare Dataset
data_tensor = make_dataset(
    data_1m, data_1d,
    using_column=feature_column, using_column_1d=feature_column_1d,
    window_size=240, window_size_1d=60,
    entry_pos_list=entry_pos_list, patience_list=patience_list,
    use_1d_data=True
)

dataset_1m = []
dataset_1d = []
skip_data_cnt = 0
for data in data_tensor:
    if len(data[0]) == 240 and len(data[1]) == 60:
        dataset_1m.append(torch.from_numpy(data[0]).unsqueeze(dim=0))
        dataset_1d.append(torch.from_numpy(data[1]).unsqueeze(dim=0))
    else:
        skip_data_cnt += 1
dataset_1m = torch.cat(dataset_1m, dim=0)
dataset_1d = torch.cat(dataset_1d, dim=0)
dataset_1m = replace_nan_with_zero(dataset_1m)
dataset_1d = replace_nan_with_zero(dataset_1d)


100%|██████████| 100000/100000 [00:10<00:00, 9944.72it/s]


In [3]:
# Evolution Setup
device = 'cuda:0'
group = 100
prescriptor = Prescriptor(
    basic_block=None, 
    base_small_input_dim=19, 
    base_large_input_dim=19,
    base_hidden_dim=24, 
    base_output_dim=16, 
    after_input_dim=19, 
    after_hidden_dim=32, 
    after_output_dim=6, 
    num_blocks=group,
    nhead=4,
    dim_feedforward=24*4,
    dropout=0.1,
    small_max_seq_length=240,
    large_max_seq_length=60
).to(device)

total_param = sum(p.numel() for p in prescriptor.parameters())
print(f"Total parameters: {total_param}")

selection = RouletteSelection(elite_num=20, parents_num=40, minimize=False)
crossover = UniformCrossover()
mutation = AddNormalMutation(mut_prob=0.2)
evolution = Evolution(
    prescriptor=prescriptor,
    selection=selection,
    crossover=crossover,
    mutation=mutation
)
init_prescriptor = deepcopy(prescriptor)

Total parameters: 3154800


In [4]:
logits = inference(dataset_1m, dataset_1d, prescriptor, device)
probs = []
for logit in logits:
    logit = torch.stack(logit, dim=0)
    probs.append(logit)
probs = torch.concat(probs, dim=1)
probs = probs.squeeze(dim=2)
profit = fitness_fn(prescriptor, data_1m, probs, skip_data_cnt, 100, 240, device=device)

100%|██████████| 100000/100000 [00:20<00:00, 4917.77it/s]


In [5]:
evolution.evolve(torch.from_numpy(profit))
elite_idx = evolution.selection.elite_idx()

In [6]:
device = 'cuda:0'
new_logits = inference(dataset_1m, dataset_1d, prescriptor, device)
new_probs = []
for logit in new_logits:
    logit = torch.stack(logit, dim=0)
    new_probs.append(logit)
new_probs = torch.concat(new_probs, dim=1)
new_probs = new_probs.squeeze(dim=2)
# new_new_profit = fitness_fn(prescriptor, data_1m, probs, skip_data_cnt, 100, 240, device=device)

In [7]:
new_profit = fitness_fn(prescriptor, data_1m, new_probs, skip_data_cnt, 100, 240, device=device)

100%|██████████| 100000/100000 [00:18<00:00, 5427.40it/s]


In [8]:
new_profit[:20]

array([951.80009409, 303.39304627, 199.05894845, 179.57905019,
       126.55143412, 108.1272874 ,  89.68259968,  58.4756267 ,
        44.08511287,  27.11453519,  23.3225461 ,  21.85541802,
        11.38198837,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [9]:
profit[elite_idx]

array([951.80009409, 303.39304627, 199.05894845, 179.57905019,
       126.55143412, 108.1272874 ,  89.68259968,  58.4756267 ,
        44.08511287,  27.11453519,  23.3225461 ,  21.85541802,
        11.3819804 ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])